In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
img=cv.imread("./sample2.png")
tmp=img.copy()
tmp1=img.copy()
plt.figure(figsize=(12,8))
# plt.subplot(2,2,1)
# b=img[:,:,0]
# plt.imshow(b,cmap="gray")
# plt.title("b")

# plt.subplot(2,2,2)
# g=img[:,:,1]
# plt.imshow(g,cmap="gray")
# plt.title("g")

# plt.subplot(2,2,3)
# r=img[:,:,2]
# plt.imshow(r,cmap="gray")
# plt.title("r")

# plt.subplot(2,2,4)
# sub=cv.subtract(b,r)
# sub=cv.subtract(sub,g)
# plt.imshow(sub,cmap="gray")
# plt.title("sub")

# plt.show()

gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret,thresh=cv.threshold(gray,150,255,cv.THRESH_BINARY)
plt.subplot(2,2,1)
plt.imshow(thresh,cmap="gray")
plt.title("org")

kernel=np.ones((3,3),np.uint8)
plt.subplot(2,2,2)
erosion = cv.erode(thresh, kernel, iterations = 1)
test=cv.dilate(erosion,kernel,iterations = 1)
test=cv.erode(test, kernel, iterations = 1)
plt.imshow(erosion,cmap="gray")
plt.title("ero")

plt.subplot(2,2,3)
dilation=cv.dilate(thresh,kernel,iterations = 1)
plt.imshow(dilation,cmap="gray")
plt.title("dil")

plt.subplot(2,2,4)
dilation=cv.dilate(test,kernel,iterations=9)
dilation=cv.erode(dilation, kernel, iterations = 19)
plt.imshow(dilation,cmap="gray")
plt.title("dil2")

plt.show()


预处理

In [ ]:
# contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
# cv.drawContours(tmp,contours,-1,(0,0,255),2)
# cv.imshow("contours",img)
# cv.waitKey(0)
# cv.destroyAllWindows()

筛选：面积（轮廓面积，边界矩形）、长宽比、拟合椭圆的角度、

In [ ]:
# contours,hierarchy=cv.findContours(dilation,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
# cnt=0
# max=0
# save=0
# for i in contours:
#     area=cv.contourArea(i)
#     if area>max:
#         save=cnt
#         max=area
#     cnt+=1

# rect = cv.minAreaRect(contours[save])
# box = cv.boxPoints(rect)
# box = np.int0(box)


# cv.circle(tmp1,(int(rect[0][0]),int(rect[0][1])),2,(0,0,255),-1)
# cv.drawContours(tmp1,[box],0,(0,0,255),2)  
# cv.imshow("result",tmp1)
# cv.waitKey(0)
# cv.destroyAllWindows()


边界矩形

In [ ]:
contours,hierarchy=cv.findContours(dilation,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
cnt=0
max=0
save=0
for i in contours:
    area=cv.contourArea(i)
    if area>max:
        save=cnt
        max=area
    cnt+=1


(x,y),radius = cv.minEnclosingCircle(contours[save])
center = (int(x),int(y))
radius = int(radius)
cv.circle(tmp1,center,radius,(0,0,255),2)
cv.circle(tmp1,center,2,(0,0,255),-1)
cv.imshow("result",tmp1)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# for cnt in contours:
#     x,y,w,h=cv.boundingRect(cnt)
#     font=cv.FONT_HERSHEY_SIMPLEX
#     cv.putText(img,f'{int(w/h)}',(w,y),font,1,(0,0,255),1,cv.LINE_AA)

#     cv.rectangle(tmp,(x,y),(x+w,y+h),(0,0,255))

旋转矩形

In [ ]:
# for cnt in contours:
#     rect = cv.minAreaRect(cnt)
#     box = cv.boxPoints(rect)
#     box = np.int0(box)
#     cv.drawContours(img,[box],0,(0,0,255),2)

视频处理

In [ ]:
cap = cv.VideoCapture('./video.MP4')

while cap.isOpened():
    ret, frame = cap.read()
    frame= cv.resize(frame, (0, 0), fx=0.3, fy=0.3, interpolation=cv.INTER_CUBIC)
    # 如果正确读取帧，ret为True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    ret,thresh=cv.threshold(gray,230,255,cv.THRESH_BINARY)
    # thresh=cv.Canny(thresh,180,255)
    kernel=np.ones((5,5),np.uint8)
    dilation=cv.dilate(thresh,kernel,iterations=1)
    erosion=cv.erode(dilation,kernel,iterations=3)
    dilation=cv.dilate(erosion,kernel,iterations=3)
    erosion=cv.erode(dilation,kernel,iterations=2)

    contours,hierarchy=cv.findContours(dilation,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    cnt=0
    max=0
    save=0
    for i in contours:
        area=cv.contourArea(i)
        if area>max:
            save=cnt
            max=area
        cnt+=1
    if contours:
        (x,y),radius = cv.minEnclosingCircle(contours[save])
        center = (int(x),int(y))
        radius = int(radius)
        cv.circle(frame,center,radius,(0,0,255),2)
        cv.circle(frame,center,2,(0,0,255),-1)
    cv.imshow("gray",erosion)  
    cv.imshow("frame",frame)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()